# AgriSmart

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
import joblib

In [2]:
DATASET_DIR = "./PlantVillage/" 

# Model parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32

In [3]:
# Prepare data using ImageDataGenerator with augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
print("\n--- Loading Data ---")
train_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)


--- Loading Data ---
Found 16516 images belonging to 15 classes.


In [5]:
val_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

Found 4122 images belonging to 15 classes.


In [6]:
# Verify the classes
class_names = list(train_data.class_indices.keys())
print("\nClasses found:", class_names)
NUM_CLASSES = len(class_names)


Classes found: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [7]:

# ## 3. Build and Compile the CNN Model
print("\n--- Building Model ---")
model = Sequential([
    keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    keras.layers.Conv2D(32, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(NUM_CLASSES, activation="softmax")
])


--- Building Model ---


In [8]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,575 (12.61 MB)

 Trainable params: 3,306,575 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# ## 4. Train the Model
print("\n--- Starting Model Training ---")
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10  # You can increase this later for better accuracy
)
print("\nTraining finished successfully!")


--- Starting Model Training ---


C:\Users\Akshat\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 391s 750ms/step - accuracy: 0.2925 - loss: 2.1695 - val_accuracy: 0.5774 - val_loss: 1.3453
Epoch 2/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 268s 518ms/step - accuracy: 0.5514 - loss: 1.3995 - val_accuracy: 0.6439 - val_loss: 1.0434
Epoch 3/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 199s 385ms/step - accuracy: 0.6223 - loss: 1.1372 - val_accuracy: 0.7310 - val_loss: 0.7873
Epoch 4/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 192s 371ms/step - accuracy: 0.6683 - loss: 0.9860 - val_accuracy: 0.7686 - val_loss: 0.7063
Epoch 5/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 196s 378ms/step - accuracy: 0.6990 - loss: 0.9191 - val_accuracy: 0.7562 - val_loss: 0.7091
Epoch 6/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 185s 357ms/step - accuracy: 0.7128 - loss: 0.8643 - val_accuracy: 0.8125 - val_loss: 0.5514
Epoch 7/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 181s 350ms/step - accuracy: 0.7478 - loss: 0.7507 - val_accuracy: 0.8503 - val_loss: 0.4709
Epoch 8/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 218s 421ms/step - accuracy: 0.7754 -

In [10]:

# ## 5. Save the Final Model and Encoder
print("\n--- Saving Artifacts ---")

# Save the trained model
# Using the .keras format is recommended over .h5
model.save("disease_model.keras")
print("✅ Model saved as disease_model.keras")


--- Saving Artifacts ---
✅ Model saved as disease_model.keras


In [11]:

# Create and save the label encoder
disease_label_encoder = LabelEncoder()
disease_label_encoder.fit(class_names)
joblib.dump(disease_label_encoder, 'disease_label_encoder.pkl')
print("✅ Label encoder saved as disease_label_encoder.pkl")

✅ Label encoder saved as disease_label_encoder.pkl
